## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_Database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pevek,RU,69.7008,170.3133,-11.76,100,56,1.72,broken clouds
1,1,Havelock,US,34.8791,-76.9013,57.06,41,1,5.01,clear sky
2,2,Ushuaia,AR,-54.8000,-68.3000,56.86,47,20,10.36,few clouds
3,3,Lavrentiya,RU,65.5833,-171.0000,-12.69,88,7,8.63,clear sky
4,4,Bengkulu,ID,-3.8004,102.2655,75.60,87,100,7.43,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Bengkulu,ID,-3.8004,102.2655,75.60,87,100,7.43,overcast clouds
7,7,Mahebourg,MU,-20.4081,57.7000,77.29,73,20,2.30,few clouds
8,8,Nacozari,MX,25.3366,-108.2574,77.11,42,98,4.76,overcast clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,76.96,89,92,7.00,overcast clouds
13,13,Conakry,GN,9.5380,-13.6773,87.87,70,20,9.22,few clouds
15,15,Bossangoa,CF,6.4926,17.4552,84.56,33,72,3.29,broken clouds
24,24,Salta,AR,-24.7859,-65.4117,88.09,32,40,7.00,scattered clouds
31,31,Rikitea,PF,-23.1203,-134.9692,76.19,77,5,23.51,light rain
34,34,San Cristobal,VE,7.7669,-72.2250,78.98,42,99,7.20,overcast clouds
38,38,Richards Bay,ZA,-28.7830,32.0377,76.93,78,38,4.83,light rain


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
cleaned_cities_df = preferred_cities_df.dropna()

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cleaned_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bengkulu,ID,75.60,overcast clouds,-3.8004,102.2655,
7,Mahebourg,MU,77.29,few clouds,-20.4081,57.7000,
8,Nacozari,MX,77.11,overcast clouds,25.3366,-108.2574,
9,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,
13,Conakry,GN,87.87,few clouds,9.5380,-13.6773,
15,Bossangoa,CF,84.56,broken clouds,6.4926,17.4552,
24,Salta,AR,88.09,scattered clouds,-24.7859,-65.4117,
31,Rikitea,PF,76.19,light rain,-23.1203,-134.9692,
34,San Cristobal,VE,78.98,overcast clouds,7.7669,-72.2250,
38,Richards Bay,ZA,76.93,light rain,-28.7830,32.0377,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# Find rows with no Hotel Name
hotel_df[hotel_df['Hotel Name'] == ''].index

Int64Index([8, 171, 188, 196, 197, 264, 286, 326, 444, 555], dtype='int64')

In [40]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.drop(hotel_df.loc[hotel_df['Hotel Name']==''].index, inplace=True)

In [41]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bengkulu,ID,75.60,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
7,Mahebourg,MU,77.29,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
9,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
13,Conakry,GN,87.87,few clouds,9.5380,-13.6773,Riviera Royal Hotel
15,Bossangoa,CF,84.56,broken clouds,6.4926,17.4552,LA PERLE DE BOSTON
24,Salta,AR,88.09,scattered clouds,-24.7859,-65.4117,Hotel Alejandro I
31,Rikitea,PF,76.19,light rain,-23.1203,-134.9692,People ThankYou
34,San Cristobal,VE,78.98,overcast clouds,7.7669,-72.2250,Posada Villaven C.A.
38,Richards Bay,ZA,76.93,light rain,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
43,Poum,NC,80.51,few clouds,-20.2333,164.0167,Gîte kajeon


In [44]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [50]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))